# Testing and Prediction Request to Model Serving

In this section, the testing and prediction request process is carried out to the model serving using an HTTP request from the model that has been successfully deployed using the cloud platform called Railway. This process sets an endpoint URL that becomes the main API to make requests in the form of an HTTP request by calling the model serving hosted by TF-Serving. The URL is `https://mlops-disaster-tweets-production.up.railway.app/v1/models/tweets-model:predict` and is designed to request predictions from the model.

Based on the machine learning pipeline process that was created, it is known that the format of the data received by the model is TFRecord, while the prediction request process in the production environment must be in the form of a JSON string (Javascript Object Notation). Some characters in TFRecord data can be a problem when included directly in JSON because they are considered invalid. Therefore, a coding technique using Base64 encoding is used, which generally represents binary data, so that TFRecord binary data can be safely included in a JSON string without the risk of invalid characters occurring and can be accepted by the model.

Here is the code to run the testing and prediction request to the model serving.

### Import Dependencies

In [1]:
import requests
import tensorflow as tf
import base64

### URL TF-Serving

In [16]:
# URL for TF-Serving
serving_url = "https://mlops-disaster-tweets-production.up.railway.app/v1/models/tweets-model:predict"

### Input String Tweet Text Example and ID Tweet

In [43]:
# input string text
id_value = 1
input_string = "Just happened a terrible car crash"

### Data Preprocessing (Encoded the serializede example from TFRecord format)

In [44]:
# create tf.train.Example from the input string
example = tf.train.Example(features=tf.train.Features(feature={
    'id': tf.train.Feature(int64_list=tf.train.Int64List(value=[id_value])),
    'text': tf.train.Feature(bytes_list=tf.train.BytesList(value=[input_string.encode()]))
}))
tfrecord_example = example.SerializeToString()

# base64 encode the serialized example
b64_encoded_example = base64.b64encode(tfrecord_example).decode("utf-8")

# prepare the input example in the correct format
input_example = {"b64": b64_encoded_example}

### Make Prediction Request

In [45]:
print("Tweet:", input_string)

# send prediction request to TF-Serving
response = requests.post(serving_url, json={"instances": [input_example]})

# check the response
if response.status_code == 200:
    result = response.json()
    model_output = result["predictions"][0][0]
    print("\nModel output:", model_output)

    # check if the model output is spam (assuming threshold is 0.75)
    tweet_threshold = 0.75
    if model_output > tweet_threshold:
        print("Prediction: Disaster tweet")
    else:
        print("Prediction: Not Disaster tweet (just normal tweet)")
else:
    print("Error:", response.status_code, response.content)

Tweet: Just happened a terrible car crash

Model output: 0.914532244
Prediction: Disaster tweet


Based on the output results of the prediction request, it was found that the example tweet used for testing was a Disaster Tweet type tweet with a model output of around 0.91. This result shows that the model works quite well in classifying tweet sentences.